In [ ]:
# !pip install requests

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import requests


In [ ]:
def download_image(image_url:str, save_path:str):
    """
    Download an image from the given URL and save it to the specified path.

    Parameters:
    - image_url (str): The URL of the image.
    - save_path (str): The local path to save the downloaded image.

    Returns:
    - bool: True if the image is downloaded successfully, False otherwise.
    """
    try:
        # Sending a GET request to the image URL
        response = requests.get(image_url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Save the image to the specified local file
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print("Image downloaded successfully.")
            return True
        else:
            print(f"Failed to download image. Status code: {response.status_code}")
            return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False



In [ ]:
API_KEY = 'wxwa149dl68wxdocc'
IMAGE_FILE_PATH = '/content/sesi.jpeg'
base_name=os.path.basename(image_path)
(file_Name,ext)=os.path.splitext(base_name)
parent_folder_path = "/content/outputs"
os.makedirs(parent_folder_path, exist_ok=True)
def create_task():
    assert os.path.exists(IMAGE_FILE_PATH), f'Error: File({IMAGE_FILE_PATH}) does not exist.'

    headers = {'X-API-KEY': API_KEY}
    data = {'sync': '0'}
    files = {'image_file': open(IMAGE_FILE_PATH, 'rb')}
    url = 'https://techhk.aoscdn.com/api/tasks/visual/scale'

    # Create a task
    response = requests.post(url, headers=headers, data=data, files=files)

    task_id = None
    response_json = response.json()
    if 'status' in response_json and response_json['status'] == 200:
        result_tag = 'failed'
        if 'data' in response_json:
            response_json_data = response_json['data']
            if 'task_id' in response_json_data:
                result_tag = 'successful'
                task_id = response_json_data['task_id']
        print(f'Result of created task({result_tag}): {response_json}')
    else:
        # request failed, log the details
        print(f'Error: Failed to create task,{response.text}')
    return task_id

# get the task result
# 1、"The polling interval is set to 1 second."
# 2、"The polling time is around 30 seconds."
def polling_task_result(task_id,time_out=30):
    headers = {'X-API-KEY': API_KEY}
    url = f'https://techhk.aoscdn.com/api/tasks/visual/scale/{task_id}'
    for i in range(time_out):
        if i != 0:
            time.sleep(1)
        response = requests.get(url, headers=headers)
        response_json = response.json()
        if 'status' in response_json and response_json['status'] == 200:
            if 'data' in response_json:
                response_json_data = response_json['data']
                if 'state' in response_json_data:
                    task_state = response_json_data['state']
                    if task_state == 1:
                        # task success
                        print(f'Result(successful): {response_json}')
                        image_url = response_json['data']['image']
                        print("Image URL:", image_url)
                        save_path = f'/content/outputs/{file_Name}.png'
                        download_image(image_url, save_path)
                        break
                    elif task_state < 0:
                        # Task exception, logging the error.
                        # You can choose to continue the loop with 'continue' or break the loop with 'break'
                        print(f'Result(failed): {response_json}')
                        break
            print(f'Result(polling): {response_json}')
            if i == time_out-1:
                # Timeout, log the details, and search for support from the picwish service.
                print('Error: Timeout while polling.')
        else:
            # Task exception, logging the error.
            # You can choose to continue the loop with 'continue' or break the loop with 'break'
            print(f'Error: Failed to get the task\'s result,{response.text}')
            break

def main():
    task_id = create_task()
    if task_id is None:
        print('Error: Failed to create task,task id is None.')
        return
    polling_task_result(task_id)

if __name__ == "__main__":
    main()